In [ ]:
%load_ext autoreload
%autoreload 2

# Clean preferred names and split into separate given and surname datasets

In [ ]:
from os.path import join

import csv
from mpire import WorkerPool
import pandas as pd
from pathlib import Path

from src.data.filesystem import glob
from src.data.normalize import normalize

In [ ]:
in_path = "s3://familysearch-names/raw/tree-preferred/"
given_out_path = "s3://familysearch-names/interim/tree-preferred-given/"
surname_out_path = "s3://familysearch-names/interim/tree-preferred-surname/"

In [ ]:
def normalize_given_and_join(name):
    return " ".join(normalize(name, False))

def normalize_surname_and_join(name):
    return " ".join(normalize(name, True))

In [ ]:
def process_file(shared, filename):
    given_out_path, surname_out_path = shared
    basename = Path(filename).stem

    df = pd.read_csv(
        filename,
        sep="|",
        compression="gzip",
        names=["name"],
        dtype={"name": str},
        na_filter=False,
        encoding="utf-8",
        encoding_errors="replace",
        on_bad_lines="warn",
        quoting=csv.QUOTE_NONE,    )

    # create separate given and surname dataframes
    given_df = df[["name"]].copy()
    surname_df = df[["name"]].copy()
    del df

    # split names into given and surname
    given_df["name"] = given_df["name"].str.replace("\^.*$", "", regex=True)
    surname_df["name"] = surname_df["name"].str.replace("^.*\^", "", regex=True)

    # filter out non-latin names
    given_df = given_df[
        given_df["name"].str.endswith("~Latn")
    ]
    surname_df = surname_df[
        surname_df["name"].str.endswith("~Latn")
    ]

    # remove ~Latn suffix and lowercase
    given_df["name"] = given_df["name"].str.replace(
        "~Latn$", "", regex=True
    ).str.lower()
    surname_df["name"] = surname_df["name"].str.replace(
        "~Latn$", "", regex=True
    ).str.lower()

    # normalize names and join the pieces back into a single space-separated string
    given_df["name"] = given_df["name"].map(normalize_given_and_join)
    surname_df["name"] = surname_df["name"].map(normalize_surname_and_join)

    # write files
    given_df.to_parquet(
        join(given_out_path, basename) + ".parquet", engine="pyarrow", compression="snappy"
    )
    surname_df.to_parquet(
        join(surname_out_path, basename) + ".parquet", engine="pyarrow", compression="snappy"
    )

In [ ]:
# process files
filenames = glob(join(in_path,"*.gz"))
with WorkerPool(shared_objects=(given_out_path, surname_out_path)) as pool:
    pool.map(process_file, filenames, progress_bar=True)